In [55]:
import numpy as np
from scipy.fft import dct, fft

def compare_dct_dft():
    # Define a sample signal (N=4)
    x = np.array([1.0, 2.0, 3.0, 4.0])
    N = len(x)
    print(f"Original Signal x: {x}\n")

    # --- CASE 1: DCT-I ---
    # Equivalent to DFT of a (2N-2) periodic extension
    # Extension: [x0, x1, x2, x3, x2, x1]
    dct_1 = dct(x, type=1)
    
    x_ext_1 = np.concatenate([x, x[1:-1][::-1]])
    dft_1 = np.real(fft(x_ext_1))[:N]
    
    print("DCT-I vs DFT (2N-2 Extension)")
    print(f"DCT-I: {dct_1}")
    print(f"DFT-1: {dft_1}")
    print(f"Match: {np.allclose(dct_1, dft_1)}\n")

    # --- CASE 2: DCT-II ---
    # Equivalent to Real part of 2N DFT of [x0...xN-1, xN-1...x0] 
    # multiplied by a phase shift factor
    dct_2 = dct(x, type=2)
    
    x_ext_2 = np.concatenate([x, x[::-1]]) # [1, 2, 3, 4, 4, 3, 2, 1]
    dft_val_2 = fft(x_ext_2)[:N]
    # Apply phase shift e^(-j*pi*k / 2N)
    k = np.arange(N)
    dft_2 = np.real(dft_val_2 * np.exp(-1j * np.pi * k / (2 * N)))
    
    print("DCT-II vs DFT (2N Extension + Phase Shift)")
    print(f"DCT-II: {dct_2}")
    print(f"DFT-II: {dft_2}")
    print(f"Match: {np.allclose(dct_2, dft_2)}\n")


    # --- CASE 3: DCT-III ---
    # Equivalent to a DFT where the input has a phase shift applied
    dct_3 = dct(x, type=3)
    
    # Pre-process input with phase shift
    k = np.arange(N)
    # Note: Scipy DCT-3 has specific scaling. To match raw DFT, 
    # we replicate the IDCT logic.
    x_shifted = np.zeros(2*N, dtype=complex)
    x_shifted[0] = x[0]
    x_shifted[1:N] = x[1:] * np.exp(1j * np.pi * k[1:] / (2 * N))
    x_shifted[N+1:] = np.conj(x_shifted[1:N][::-1]) # Ensure Hermitian symmetry for real output
    
    dft_3 = np.real(fft(x_shifted))[:N]
    
    # DCT-3 is the inverse of DCT-2, usually requires specific scaling comparison
    print("DCT-III vs DFT (Frequency-Shifted Logic)")
    print(f"DCT-3: {dct_3}")
    # (Note: Scaling constants may vary by implementation library)


    # --- CASE 4: DCT-IV ---
    # Equivalent to a 2N DFT with both input and output phase shifts
    dct_4 = dct(x, type=4)
    
    # Shift input
    n = np.arange(N)
    x_in_shifted = x * np.exp(-1j * np.pi * n / (2 * N))
    
    # 2N-point DFT of zero-padded shifted input
    dft_val_4 = fft(x_in_shifted, 2*N)[:N]
    
    # Shift output
    k = np.arange(N)
    dft_4 = np.real(dft_val_4 * np.exp(-1j * np.pi * (k + 0.5) / (2 * N))) * 2
    
    print("DCT-IV vs DFT (Dual Phase Shift)")
    print(f"DCT-IV: {dct_4}")
    print(f"DFT-IV: {dft_4}")
    print(f"Match: {np.allclose(dct_4, dft_4)}")

    # Define a sample signal (N=4)
    x = np.array([1.0, 2.0, 3.0, 4.0])
    N = len(x)
    print(f"Original Signal x: {x}\n")

if __name__ == "__main__":
    compare_dct_dft()

Original Signal x: [1. 2. 3. 4.]

DCT-I vs DFT (2N-2 Extension)
DCT-I: [ 1.50000000e+01 -4.00000000e+00 -2.22044605e-16 -1.00000000e+00]
DFT-1: [ 1.50000000e+01 -4.00000000e+00 -2.22044605e-16 -1.00000000e+00]
Match: True

DCT-II vs DFT (2N Extension + Phase Shift)
DCT-II: [20.         -6.30864406  0.         -0.44834153]
DFT-II: [20.         -6.30864406  0.         -0.44834153]
Match: True

DCT-III vs DFT (Frequency-Shifted Logic)
DCT-3: [11.99962628 -9.10294322  2.61766184 -1.5143449 ]
DCT-IV vs DFT (Dual Phase Shift)
DCT-IV: [10.18159298 -9.44669561  5.01029817 -4.68956486]
DFT-IV: [10.18159298 -9.44669561  5.01029817 -4.68956486]
Match: True
Original Signal x: [1. 2. 3. 4.]



In [ ]:
# --- Test
n = 12
x0 = np.array(np.arange(1,n+1))
x1 = np.concatenate([x0, x0[0:-1][::-1]])
x_ext_0 = np.concatenate([x0, x0[1:-1][::-1]])
x_ext_1 = x1[:-1]
# Equivalent to DFT of a (2N-2) periodic extension
# x        : [x0, x1, x2, x3, x2, x1]
# Extension: [x0, x1, x2, x3, x2, x1]
print(len(x0), x0)
print(len(x1), x1)
print(len(x_ext_0), x_ext_0)
# print(x_ext_1)

dct_0 = dct(x0, type=1)/n
dct_1 = dct(x1, type=1)/n/2
dft_0 = np.real(fft(x_ext_0))/n
dft_1 = np.real(fft(x_ext_1))/n

dct_0[np.abs(dct_0) < 1e-10] = 0.0
dct_1[np.abs(dct_1) < 1e-10] = 0.0
dft_0[np.abs(dft_0) < 1e-10] = 0.0
dft_1[np.abs(dft_1) < 1e-10] = 0.0

print("")
print(f"DCT-I: {dct_0}")
print(f"DCT-I: {dct_1}")
print(f"DFT-1: {dft_0}")
# print(f"DFT-1: {dft_1}")
# print(f"Match: {np.allclose(dct_1, dft_1)}\n")

12 [ 1  2  3  4  5  6  7  8  9 10 11 12]
23 [ 1  2  3  4  5  6  7  8  9 10 11 12 11 10  9  8  7  6  5  4  3  2  1]
22 [ 1  2  3  4  5  6  7  8  9 10 11 12 11 10  9  8  7  6  5  4  3  2]

DCT-I: [11.91666667 -4.11451251  0.         -0.48289686  0.         -0.1943215
  0.         -0.11775109  0.         -0.09051804  0.         -0.08333333]
DCT-I: [11.91666667  0.         -4.11451251  0.          0.          0.
 -0.48289686  0.          0.          0.         -0.1943215   0.
  0.          0.         -0.11775109  0.          0.          0.
 -0.09051804  0.          0.          0.         -0.08333333]
DFT-1: [11.91666667 -4.11451251  0.         -0.48289686  0.         -0.1943215
  0.         -0.11775109  0.         -0.09051804  0.         -0.08333333
  0.         -0.09051804  0.         -0.11775109  0.         -0.1943215
  0.         -0.48289686  0.         -4.11451251]


In [57]:
# --- CASE 2: DCT-II ---
# Equivalent to Real part of 2N DFT of [x0...xN-1, xN-1...x0] 
# multiplied by a phase shift factor
n = 12
x0 = np.array(np.arange(1,n+1))
dct_2 = dct(x0, type=2)

x_ext_2 = np.concatenate([x0, x0[::-1]]) # [1, 2, 3, 4, 4, 3, 2, 1]
dft_val_2 = fft(x_ext_2)
# Apply phase shift e^(-j*pi*k / 2N)
k = np.arange(2*n)
dft_2 = np.real(dft_val_2 * np.exp(-1j * np.pi * k / (2 * n)))

dct_2[np.abs(dct_2) < 1e-10] = 0.0
dft_2[np.abs(dft_2) < 1e-10] = 0.0

print("DCT-II vs DFT (2N Extension + Phase Shift)")
print(f"DCT-II: {dct_2}")
print(f"DFT-II: {dft_2}")
# print(f"Match: {np.allclose(dct_2, dft_2)}\n")

DCT-II vs DFT (2N Extension + Phase Shift)
DCT-II: [ 1.56000000e+02 -5.81933326e+01  0.00000000e+00 -6.30864406e+00
  0.00000000e+00 -2.14078178e+00  0.00000000e+00 -9.67194501e-01
  0.00000000e+00 -4.48341529e-01  0.00000000e+00 -1.32788522e-01]
DFT-II: [ 1.56000000e+02 -5.81933326e+01  0.00000000e+00 -6.30864406e+00
  0.00000000e+00 -2.14078178e+00  0.00000000e+00 -9.67194501e-01
  0.00000000e+00 -4.48341529e-01  0.00000000e+00 -1.32788522e-01
  0.00000000e+00  1.32788522e-01  0.00000000e+00  4.48341529e-01
  0.00000000e+00  9.67194501e-01  0.00000000e+00  2.14078178e+00
  0.00000000e+00  6.30864406e+00  0.00000000e+00  5.81933326e+01]
